In [139]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [140]:
## Trans-ancestry 

In [141]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)

In [142]:
lead <- read.table("../../ANALYSIS/lead_transancestry_3mb.txt",fill=TRUE,header=TRUE)
output_df <- merge(TRANSANCESTRY_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)

In [143]:
head(output_df)

,rsid,chromosome,position,allele1,allele2,maf,beta,se
,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,rs10000081,4,17348363,T,C,0.33229026,-0.0095197,0.00463699
2,rs10000092,4,21895517,C,T,0.38806186,0.0121143,0.00451048
3,rs1000025,6,134085319,T,C,0.28441548,-0.0111126,0.00485617
4,rs10000252,4,3216484,T,C,0.02623173,0.0314747,0.01459080
5,rs10000268,4,63606939,T,A,0.16447520,-0.0106829,0.00585867
6,rs10000289,4,96437606,A,T,0.03235335,0.0396380,0.01743640


In [144]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".z")
caviar_name <- paste0(chr_q,"_",start_q,"_",end_q,"_caviar.z")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()
caviar_df <- tmp_df%>%mutate(z=pchisq((beta/se)^2, df=1, lower=F))%>% distinct()
write.table(tmp_df,paste0("../../ANALYSIS/CAVIAR/trans/meta_z/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
write.table(caviar_df,paste0("../../ANALYSIS/CAVIAR/trans/",caviar_name),sep = " ",row.names=FALSE,col.names = FALSE,quote=FALSE)
}

In [145]:
#GTEX 

In [146]:
df_z <- list.files("../../ANALYSIS/CAVIAR/GTEx_Analysis_v8_eQTL/",pattern='*._Z',full.names = TRUE)

In [147]:
suppressMessages(suppressWarnings(for (i in 1:length(df_z))
{
    tryCatch({
name <- tail(str_split(df_z,"/|.z")[[i]],n=2)[2]
expr_name <- paste0(name,"_Zscore.txt")
z_name <- paste0(name,".z")
expr_file  <- read.table(df_z[i],header=FALSE,fill = TRUE)
colnames(expr_file) <- c("MARKERNAME","eQTL_ZSCORE")
expr_file_z <- merge(Metasoft_results,expr_file,by="MARKERNAME")%>%select(MARKERNAME,eQTL_ZSCORE)%>% distinct()
eqtl_z <- merge(TRANSANCESTRY_RESULTS,expr_file,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>% distinct()
write.table(expr_file_z,paste0("../../ANALYSIS/CAVIAR/trans/eQTL/",expr_name),sep = " ",row.names=FALSE,quote=FALSE,col.names = FALSE)
write.table(eqtl_z,paste0("../../ANALYSIS/CAVIAR/trans/expression_z/",z_name),sep = " ",row.names=FALSE,quote=FALSE)       
   }, error=function(e){})
}))

In [ ]:
###

In [148]:
### European 

In [149]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUR_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)

In [152]:
lead <- read.table("../../ANALYSIS/lead_european_3mb.txt",fill=TRUE,header=TRUE)
 output_df <- merge(EUR_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME.x,POSITION.x,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME.x,position=POSITION.x,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)

In [153]:
head(output_df)

,rsid,chromosome,position,allele1,allele2,maf,beta,se
,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,rs10000081,4,17348363,T,C,0.27695719,-0.0105171,0.00486910
2,rs10000092,4,21895517,C,T,0.33079187,-0.0116018,0.00464468
3,rs1000012,9,135703041,C,A,0.24825206,0.0101299,0.00513429
4,rs10000180,4,83899764,A,G,0.28289825,-0.0101409,0.00486157
5,rs1000025,6,134085319,T,C,0.30296044,-0.0119621,0.00503065
6,rs10000252,4,3216484,T,C,0.02621844,0.0315125,0.01477620


In [154]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".z")
caviar_name <- paste0(chr_q,"_",start_q,"_",end_q,"_caviar.z")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()
caviar_df <- tmp_df%>%mutate(z=pchisq((beta/se)^2, df=1, lower=F))%>% distinct()
write.table(tmp_df,paste0("../../ANALYSIS/CAVIAR/eur/meta_z/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
write.table(caviar_df,paste0("../../ANALYSIS/CAVIAR/eur/",caviar_name),sep = " ",row.names=FALSE,col.names=FALSE,quote=FALSE)
}

In [155]:
##GTEX

In [156]:
suppressMessages(suppressWarnings(for (i in 1:length(df_z))
{
    tryCatch({
name <- tail(str_split(df_z,"/|.z")[[i]],n=2)[2]
expr_name <- paste0(name,"_Zscore.txt")
z_name <- paste0(name,".z")
expr_file  <- read.table(df_z[i],header=FALSE,fill = TRUE)
colnames(expr_file) <- c("MARKERNAME","eQTL_ZSCORE")
expr_file_z <- merge(Metasoft_results,expr_file,by="MARKERNAME")%>%select(MARKERNAME,eQTL_ZSCORE)%>% distinct()
eqtl_z <- merge(EUR_RESULTS,expr_file,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>% distinct()
write.table(expr_file_z,paste0("../../ANALYSIS/CAVIAR/eur/eQTL/",expr_name),sep = " ",row.names=FALSE,quote=FALSE,col.names = FALSE)
write.table(eqtl_z,paste0("../../ANALYSIS/CAVIAR/eur/expression_z//",z_name),sep = " ",row.names=FALSE,quote=FALSE)       
   }, error=function(e){})
}))

In [ ]:
### 